In [2]:
import os 
import glob
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from qmplot import manhattanplot

os.chdir("/mnt/BioHome/jreyna/jreyna/projects/dchallenge/")
outdir = 'results/main/basic_stats/'
os.makedirs(outdir, exist_ok=True)

## Loading GWAS

In [53]:
# loading GWAS samplesheet information
gwas_meta = pd.read_table('config/gwas.samplesheet.tsv')
gwas_meta.loc[:, 'gwas_id'] = gwas_meta['gwas_id'].str.replace('#', '').values.tolist()
gwas_meta

In [66]:
# loading GWAS SNP information
gwas_fn = 'results/main/2021_Nikhil_eQTL/Data/T1D_GWAS/'
gwas_fn += '*/GRCh38/GWAS_input_colocalization_pval_lt_5eMinus8.txt'
gwas_glob = glob.glob(gwas_fn)

data = []
for fn in gwas_glob:
    df = pd.read_table(fn, ' ')
    study = fn.split('/')[5]
    
    if study in gwas_meta.gwas_id.values:
        df['study'] = study
        data.append(df)

all_df = pd.concat(data)
all_df['sid'] = all_df['CHR'].str.replace('chr', '') + ':' +  all_df['POS'].astype(str)

## Unique SNPs

In [68]:
study_grps = all_df.groupby('study')

In [74]:
uniq_snps = study_grps.sid.nunique().to_frame()
uniq_snps = uniq_snps.merge(gwas_meta[['gwas_id', 'authorship']], left_index=True, right_on='gwas_id')
uniq_snps = uniq_snps.loc[:, ['authorship', 'gwas_id', 'sid']]

In [77]:
uniq_snps

,authorship,gwas_id,sid
3,"Onengut-Gumuscu et al., 2015",T1D_25751624,1722
2,"Forgetta et al., 2020",T1D_32005708,21097
0,"Chiou et al., 2021",T1D_34012112_Gaulton,45798
1,"Sakaue et al., 2021",T1D_34594039_GCST90018925,20419


## Getting the list of shared SNPs between all methods (if any)

In [83]:
i = 0 
for grp, study_df in study_grps:
    
    if i == 0:
        shared = set(study_df.sid.tolist())
    else:
        shared = shared.intersection(study_df.sid.tolist())

    i += 1 

In [85]:
len(shared)

191